In [ ]:
# send to database
# unique.to_sql('final_unique', engine)

In [ ]:
stats[(stats.year==2022)].sort_values('event_completed')['event_completed'].unique()

In [2]:
import warnings
warnings.filterwarnings('ignore')

##  IMPORT DEPENDENCIES

from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px

# files

file_one = "..//practice_sources/stats.csv"
file_one_df = pd.read_csv(file_one)
file_three = "..//practice_sources/dg_rankings.csv"
file_three_df = pd.read_csv(file_three)

In [35]:
# cleanup 'finish_pos'
stats = file_one_df.copy()
stats['fin_text'] = pd.to_numeric(stats['fin_text'].str.replace("T",""), errors='coerce')
stats.rename(columns={'fin_text':"finish_pos"}, inplace=True) 

# make unique event identifier just in case it becomes handy later (concat year and event_id)
# add score to par column
stats['unique_event_id'] = stats['season'].astype(str) + stats['event_id'].astype(str)
stats['score_to_par'] = stats['round_score'] - stats['course_par']
stats.drop(stats[stats.round_score < 40].index, inplace=True)

# add player world rankings, current datagolf rankings, and skill estimate
dg_rankings = file_three_df.copy()
temp_ranks = dg_rankings[['player_name', 'owgr_rank', 'datagolf_rank', 'dg_skill_estimate']]
stats = pd.merge(stats, temp_ranks, on='player_name')

non_stat_cols = ['event_name','unique_event_id','event_completed','player_name','round_num','round_score','finish_pos','datagolf_rank']

# leaderboard thru 3 rounds of all tournaments
temp = stats[(stats.round_num < 4) & (stats.season > 2017)][non_stat_cols].sort_values(['event_completed','player_name','round_num','round_score'])
temp['cum_sum'] = temp.groupby(['player_name','unique_event_id'])['round_score'].cumsum(axis=0)
leaderboard_after_3 = temp[temp.round_num==3].sort_values(['unique_event_id', 'cum_sum']) # keeping for leaderboard thru 3 rounds

# # r4_delta column 
temp = leaderboard_after_3.groupby('unique_event_id')[['unique_event_id','cum_sum']].min().rename(columns={'cum_sum':'cum_sum_min'}).reset_index(drop=True)
leaderboard_deltas_after_3 = pd.merge(leaderboard_after_3,temp,on='unique_event_id')
leaderboard_deltas_after_3['r4_delta'] = leaderboard_deltas_after_3.cum_sum - leaderboard_deltas_after_3.cum_sum_min

# data golf rankings bins by 100
bins100 = [0, 100, 200, 300, 400, 500]
labels100 = ['1-100', '101-200', '201-300', '301-400', '401-500']
bins40 = [0, 40, 80, 120, 160, 200, 240, 280, 320, 360, 400, 440, 500]
labels40 = ['1-40', '41-80', '81-120', '121-160', '161-200', 
            '201-240', '241-280', '281-320', '321-360', '361-400', 
            '400-440', '441-500']
stats['bin_40'] = pd.cut(stats['datagolf_rank'], bins=bins40, labels=labels40)
stats['bin_100'] = pd.cut(stats['datagolf_rank'], bins=bins100, labels=labels100)
leaderboard_deltas_after_3['bin_100'] = pd.cut(leaderboard_deltas_after_3['datagolf_rank'], bins=bins100, labels=labels100)
leaderboard_deltas_after_3['bin_40'] = pd.cut(leaderboard_deltas_after_3['datagolf_rank'], bins=bins40, labels=labels40)

# remove winners, whittle down to 2 strokes delta, add loser_key
losers_df = leaderboard_deltas_after_3[(leaderboard_deltas_after_3.r4_delta <= 2) & (leaderboard_deltas_after_3.finish_pos > 1)].reset_index(drop=True)
losers_df['loser_key'] = list(zip(losers_df.player_name.astype(str), losers_df.unique_event_id.astype('category')))

# prior and post dates for search
losers_df.event_completed = pd.to_datetime(losers_df.event_completed)
losers_df['prior_16'] = losers_df['event_completed'] - pd.Timedelta(weeks=16)
losers_df['prior_24'] = losers_df['event_completed'] - pd.Timedelta(weeks=24)
losers_df['prior_32'] = losers_df['event_completed'] - pd.Timedelta(weeks=32)

losers_df['post_2'] = losers_df['event_completed'] + pd.Timedelta(weeks=2)
losers_df['post_4'] = losers_df['event_completed'] + pd.Timedelta(weeks=4)
losers_df['post_8'] = losers_df['event_completed'] + pd.Timedelta(weeks=8)

# # remove now un-needed columns and fix dtypes
losers_df.drop(columns=['round_score','round_num','cum_sum','cum_sum_min'], axis=1, inplace=True)
#losers_df.event_completed = pd.to_datetime(losers_df.event_completed)
losers_df.finish_pos = losers_df.finish_pos.astype('int64')
stats.event_completed = pd.to_datetime(stats.event_completed)

data = losers_df.copy() # the 554 instances where a player was leading or within 2 strokes going into round 4 and lost

In [36]:
data.tail(2)

,event_name,unique_event_id,event_completed,player_name,finish_pos,datagolf_rank,r4_delta,bin_100,bin_40,loser_key,prior_16,prior_24,prior_32,post_2,post_4,post_8
552,Arnold Palmer Invitational presented by Master...,20229,2022-03-06,"Horschel, Billy",2,21,0,1-100,1-40,"(Horschel, Billy, 20229)",2021-11-14,2021-09-19,2021-07-25,2022-03-20,2022-04-03,2022-05-01
553,Arnold Palmer Invitational presented by Master...,20229,2022-03-06,"Hovland, Viktor",2,15,1,1-100,1-40,"(Hovland, Viktor, 20229)",2021-11-14,2021-09-19,2021-07-25,2022-03-20,2022-04-03,2022-05-01


In [39]:
drop_cols = ['sg_putt', 'sg_arg', 'sg_app', 'sg_ott',
           'sg_t2g','driving_dist', 'driving_acc', 'gir',
           'scrambling', 'prox_rgh', 'prox_fw','owgr_rank', 
           'datagolf_rank', 'dg_skill_estimate','course_num',
           'course_par', ]

losers_rounds = []

for i in np.arange(0, len(data)):
    
    guy = data.player_name[i]
    prior_16 = str(data.prior_16[i])
    prior_24 = str(data.prior_24[i])
    prior_32 = str(data.prior_32[i])
    post_2 = str(data.post_2[i])
    post_4 = str(data.post_4[i])
    post_8 = str(data.post_8[i])
    event = str(data.event_completed[i])
    unique_event = str(data.unique_event_id[i])

    before_guy = (stats
                  [(stats.player_name == guy) & 
                   (stats.event_completed > prior_16) &
                   (stats.event_completed < event)
                  ]
                 ).drop(columns=drop_cols)
    before_guy['target'] = 0

    after_guy = (stats
                  [(stats.player_name == guy) & 
                   (stats.event_completed > event) &
                   (stats.event_completed < post_8)
                  ]
                 ).drop(columns=drop_cols)
    after_guy['target'] = 1

    # concat before and after rounds
    one_guy = pd.concat([before_guy, after_guy]).sort_values('event_completed').reset_index(drop=True)
#     one_guy['loser_key'] = list(zip(one_guy.player_name.astype(str), one_guy.unique_event_id.astype('category')))
    one_guy['loser_key'] = f'({guy}, {unique_event})'
    losers_rounds.append(one_guy)


In [54]:
list_name = losers_rounds
df = pd.DataFrame (list_name, columns = one_guy.columns)

ValueError: Shape of passed values is (554, 1), indices imply (554, 19)

In [ ]:
fig = px.scatter(df.groupby('event_completed')[['score_to_par','target']].mean(),
            y='score_to_par',
             color='target')
fig.show()

In [15]:
# CHART INPUTS ##############################################

# stroke_delta chose 0, 1, or 2
# weeks_prior chose any amount
# weeks_after chose any amount
# rank_bin chose 'bin_40' or 'bin_100'
# min_instances chose any number > 0

stroke_delta = int(0)
weeks_prior = int(26)
weeks_after = int(4)
rank_bin = 'bin_100'
min_instances = int(2)

#############################################################

chart_data = data[data.r4_delta <= stroke_delta].reset_index(drop=True)

chart_data['prior_date'] = chart_data['event_completed'] - pd.Timedelta(weeks=weeks_prior)
chart_data['post_date'] = chart_data['event_completed'] + pd.Timedelta(weeks=weeks_after)

stats_cols = ['round_score','sg_putt', 'sg_arg', 'sg_app', 'sg_ott',
       'sg_t2g', 'sg_total', 'driving_dist', 'driving_acc', 'gir',
       'scrambling', 'prox_rgh', 'prox_fw']

#######

losers_rounds = []

for i in np.arange(0, len(chart_data)):

    guy = chart_data.player_name[i]
    prior = str(chart_data.prior_date[i])
    event = str(chart_data.event_completed[i])
    post = str(chart_data.post_date[i])
    
    rounds_before = round(stats
                          [(stats.player_name == guy) & (stats.event_completed > prior) & (stats.event_completed < event)]
                          .groupby('player_name', as_index=False)
                          [stats_cols]
                          .mean(),2)
    
    rounds_after = round(stats
                         [(stats.player_name == guy) & (stats.event_completed < post) & (stats.event_completed > event)]
                         .groupby('player_name', as_index=False)
                         [stats_cols]
                         .mean(),2) 

    # add post_ prefix to columns headers
    rounds_after.columns = 'post_' + rounds_after.columns
    rounds_after.rename(columns={'post_player_name':'player_name'}, inplace=True)
    
    # merge players before and after rounds together and send to list
    all_rounds = pd.merge(rounds_before,rounds_after, on='player_name')
    losers_rounds.append(all_rounds)
    
########

# concat before and after rounds
custom_chart = pd.concat(losers_rounds).reset_index(drop=True)

# add datagolf rankings columns
temp_ranks = dg_rankings[['player_name','datagolf_rank']]
custom_chart = pd.merge(custom_chart, temp_ranks, on='player_name')

# bring in 'count', wins, and 'datagolf_rank', and 'rank_bin'
temp_count = custom_chart.groupby('player_name',as_index=False)['sg_total'].count().sort_values('sg_total', ascending=False).rename(columns={'sg_total':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_count, on='player_name', how = 'left')
temp_bin = chart_data[['player_name',rank_bin]].drop_duplicates()#.sort_values('loser_key', ascending=False).rename(columns={'loser_key':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_bin, on='player_name', how = 'left')
temp_wins = stats[(stats.finish_pos == 1) & (stats.round_num == 4)].groupby('player_name')['finish_pos'].count().sort_values(ascending=False)
custom_chart = custom_chart.merge(temp_wins, on='player_name', how = 'left')
custom_chart.rename(columns={'finish_pos':'career_wins'},inplace=True)
custom_chart['career_wins'] = custom_chart['career_wins'].fillna(0)

temp_m = custom_chart.groupby('player_name',as_index=False)[['sg_total', 'post_sg_total']].mean()
custom_chart = custom_chart.merge(temp_m, on='player_name', how = 'left').rename(columns = 
                                                                                 {'sg_total_x':'sg_total_round', 
                                                                                  'sg_total_y':'sg_total', 
                                                                                  'post_sg_total_x':'post_sg_total_round',
                                                                                  'post_sg_total_y':'post_sg_total'}
                                                                                )
# calculate deltas
custom_chart['delta_sg_total'] = custom_chart['post_sg_total'] - custom_chart['sg_total']
# custom_chart['delta_sg_t2g'] = custom_chart['post_sg_t2g'] - custom_chart['sg_t2g']
# custom_chart['delta_sg_ott'] = custom_chart['post_sg_ott'] - custom_chart['sg_ott']
# custom_chart['delta_sg_app'] = custom_chart['post_sg_app'] - custom_chart['sg_app']
# custom_chart['delta_sg_arg'] = custom_chart['post_sg_arg'] - custom_chart['sg_arg']
# custom_chart['delta_sg_putt'] = custom_chart['post_sg_putt'] - custom_chart['sg_putt']

# title formatting
def format_title(title, subtitle=None, subtitle_font_size=16):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title} <b>{stroke_delta} or Less</b><br>Min Occurances: {min_instances}  /  {subtitle} {custom_chart[custom_chart['count'] >= min_instances]['delta_sg_total'].count()}  /  Player Count: {custom_chart[custom_chart['count'] >= min_instances]['player_name'].nunique()}<br>"

# graph
fig = px.scatter(round(custom_chart[custom_chart['count'] >= min_instances].sort_values(rank_bin),2),
                 x = 'sg_total',
                 y = 'post_sg_total',
                 title = format_title('STROKES BACK:', "Sample Size:"),
                 template = 'seaborn',
                 color = rank_bin,
                 size = 'count',
                 size_max = 20,
                 hover_name='player_name',
                 custom_data=['player_name', 'count','datagolf_rank','delta_sg_total','career_wins'],
                 height = 700,
                 width = 950
                )

# format grids and lines
fig.add_hline(y=0,line_width=3, line_color="Black")
fig.add_vline(x=0,line_width=3, line_color="Black")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.add_shape(type="line", x0=-1.5, y0=-1.5, x1=2.5, y1=2.5, line=dict(color="DarkRed", width=1.5, dash="dashdot"))

# format always-on labels
fig.update_layout(xaxis_title=(f"SG/Rd - {str(weeks_prior)} WEEKS PRIOR"))
fig.update_layout(yaxis_title=(f"SG/Rd - {str(weeks_after)} WEEKS AFTER"))
fig.update_layout(yaxis = dict(titlefont = dict(size=15)))
fig.update_layout(xaxis = dict(titlefont = dict(size=15)))
fig.update_layout(legend_title="Datagolf Rank")

# format hover labels
fig.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))
fig.update_traces(hovertemplate=
                    "<b>%{customdata[0]}</b> \
                    <br>Count: %{customdata[1]}</b> \
                    <br>=================</b> \
                    <br>SG Before:%{x:>20}<br>After Loss SG:%{y:>14}</b> \
                    <br><b>Change:%{customdata[3]:>23}</b> \
                    <br>=================</b> \
                    <br>Wins Since 2017:%{customdata[4]:>11}</b> \
                    <br>Data Golf Rank:%{customdata[2]:>13}")
                                                  

fig.show()

In [16]:
custom_chart.head(20)

,player_name,round_score,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total_round,driving_dist,driving_acc,...,post_scrambling,post_prox_rgh,post_prox_fw,datagolf_rank,count,bin_100,career_wins,sg_total,post_sg_total,delta_sg_total
0,"List, Luke",70.33,-0.17,0.20,0.07,0.54,0.81,0.75,304.56,0.57,...,0.64,40.45,34.85,93,1,1-100,1.0,0.7500,1.730000,0.980000
1,"Kisner, Kevin",71.56,0.21,0.05,-0.30,-0.09,-0.34,-0.24,280.79,0.68,...,0.66,48.65,29.27,71,1,1-100,2.0,-0.2400,0.460000,0.700000
2,"Schauffele, Xander",70.57,0.07,-0.00,0.46,0.29,0.74,0.81,295.07,0.62,...,0.53,52.41,29.59,3,5,1-100,7.0,1.4000,0.610000,-0.790000
3,"Schauffele, Xander",69.10,-0.22,0.35,0.64,0.05,1.03,1.20,295.91,0.65,...,0.64,71.06,37.11,3,5,1-100,7.0,1.4000,0.610000,-0.790000
4,"Schauffele, Xander",70.22,-0.07,0.20,0.79,0.63,1.63,1.51,302.35,0.64,...,0.63,32.69,28.21,3,5,1-100,7.0,1.4000,0.610000,-0.790000
5,"Schauffele, Xander",68.97,0.32,0.27,0.20,0.72,1.18,1.50,296.91,0.68,...,0.59,52.26,33.24,3,5,1-100,7.0,1.4000,0.610000,-0.790000
6,"Schauffele, Xander",68.94,0.93,0.34,0.46,0.38,1.18,1.98,297.55,0.65,...,0.64,40.02,34.93,3,5,1-100,7.0,1.4000,0.610000,-0.790000
7,"Spieth, Jordan",69.77,-0.25,0.29,0.45,0.48,1.22,1.23,291.92,0.67,...,0.78,35.91,28.38,16,4,1-100,5.0,0.7300,1.515000,0.785000
8,"Spieth, Jordan",69.58,0.30,0.59,1.15,0.12,1.86,2.22,294.24,0.61,...,0.62,41.78,33.22,16,4,1-100,5.0,0.7300,1.515000,0.785000
9,"Spieth, Jordan",71.27,-0.25,0.30,-0.13,-0.50,-0.33,-0.53,293.01,0.58,...,0.65,43.53,29.97,16,4,1-100,5.0,0.7300,1.515000,0.785000


In [ ]:
# CHART INPUTS ##############################################

# stroke_delta chose 0, 1, or 2
# weeks_prior chose any amount
# weeks_after chose any amount
# rank_bin chose 'bin_40' or 'bin_100'
# min_instances chose any number > 0

stroke_delta = int(1)
weeks_prior = int(24)
weeks_after = int(6)
rank_bin = 'bin_100'
min_instances = int(2)

#############################################################

chart_data = data[data.r4_delta <= stroke_delta].reset_index(drop=True)

chart_data['prior_date'] = chart_data['event_completed'] - pd.Timedelta(weeks=weeks_prior)
chart_data['post_date'] = chart_data['event_completed'] + pd.Timedelta(weeks=weeks_after)

stats_cols = ['round_score','sg_putt', 'sg_arg', 'sg_app', 'sg_ott',
       'sg_t2g', 'sg_total', 'driving_dist', 'driving_acc', 'gir',
       'scrambling', 'prox_rgh', 'prox_fw']

#######

losers_rounds = []

for i in np.arange(0, len(chart_data)):

    guy = chart_data.player_name[i]
    prior = str(chart_data.prior_date[i])
    event = str(chart_data.event_completed[i])
    post = str(chart_data.post_date[i])
    
    rounds_before = round(stats
                          [(stats.player_name == guy) & (stats.event_completed > prior) & (stats.event_completed < event)]
                          .groupby('player_name', as_index=False)
                          [stats_cols]
                          .mean(),2)
    
    rounds_after = round(stats
                         [(stats.player_name == guy) & (stats.event_completed < post) & (stats.event_completed > event)]
                         .groupby('player_name', as_index=False)
                         [stats_cols]
                         .mean(),2) 

    # add post_ prefix to columns headers
    rounds_after.columns = 'post_' + rounds_after.columns
    rounds_after.rename(columns={'post_player_name':'player_name'}, inplace=True)
    
    # merge players before and after rounds together and send to list
    all_rounds = pd.merge(rounds_before,rounds_after, on='player_name')
    losers_rounds.append(all_rounds)
    
########

# concat before and after rounds
custom_chart = pd.concat(losers_rounds).reset_index(drop=True)

# add datagolf rankings columns
temp_ranks = dg_rankings[['player_name','datagolf_rank']]
custom_chart = pd.merge(custom_chart, temp_ranks, on='player_name')

# bring in 'count', wins, and 'datagolf_rank', and 'rank_bin'
temp_count = custom_chart.groupby('player_name',as_index=False)['driving_dist'].count().sort_values('driving_dist', ascending=False).rename(columns={'driving_dist':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_count, on='player_name', how = 'left')
temp_bin = chart_data[['player_name',rank_bin]].drop_duplicates()#.sort_values('loser_key', ascending=False).rename(columns={'loser_key':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_bin, on='player_name', how = 'left')
temp_wins = stats[(stats.finish_pos == 1) & (stats.round_num == 4)].groupby('player_name')['finish_pos'].count().sort_values(ascending=False)
custom_chart = custom_chart.merge(temp_wins, on='player_name', how = 'left')
custom_chart.rename(columns={'finish_pos':'career_wins'},inplace=True)
custom_chart['career_wins'] = custom_chart['career_wins'].fillna(0)

temp_m = custom_chart.groupby('player_name',as_index=False)[['sg_total', 'post_sg_total']].mean()
custom_chart = custom_chart.merge(temp_m, on='player_name', how = 'left').rename(columns = 
                                                                                 {'sg_total_x':'sg_total_round', 
                                                                                  'sg_total_y':'sg_total', 
                                                                                  'post_sg_total_x':'post_sg_total_round',
                                                                                  'post_sg_total_y':'post_sg_total'}
                                                                                )
# calculate deltas
custom_chart['delta_sg_total'] = custom_chart['post_sg_total'] - custom_chart['sg_total']
custom_chart['delta_sg_t2g'] = custom_chart['post_sg_t2g'] - custom_chart['sg_t2g']
custom_chart['delta_sg_ott'] = custom_chart['post_sg_ott'] - custom_chart['sg_ott']
custom_chart['delta_sg_app'] = custom_chart['post_sg_app'] - custom_chart['sg_app']
custom_chart['delta_sg_arg'] = custom_chart['post_sg_arg'] - custom_chart['sg_arg']
custom_chart['delta_sg_putt'] = custom_chart['post_sg_putt'] - custom_chart['sg_putt']

def format_title(title, subtitle=None, subtitle_font_size=16):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title} <b>{stroke_delta} or Less</b><br>Min Occurances: {min_instances}  /  {subtitle} {custom_chart[custom_chart['count'] >= min_instances]['delta_sg_total'].count()}  /  Player Count: {custom_chart[custom_chart['count'] >= min_instances]['player_name'].nunique()}<br>"


fig = px.scatter(round(custom_chart[custom_chart['count'] >= min_instances].sort_values(rank_bin),2),
                 x = 'sg_total',
                 y = 'post_sg_total',
                 title = format_title('STROKES BACK:', "Sample Size:"),
                 template = 'seaborn',
                 color = rank_bin,
                 size = 'count',
                 size_max = 20,
                 hover_name='player_name',
                 custom_data=['player_name', 'count','datagolf_rank','delta_sg_total','career_wins'],
                 height = 700,
                 width = 950
                )

# format grids and lines
fig.add_hline(y=0,line_width=3, line_color="Black")
fig.add_vline(x=0,line_width=3, line_color="Black")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.add_shape(type="line", x0=-1.5, y0=-1.5, x1=2.5, y1=2.5, line=dict(color="DarkRed", width=1.5, dash="dashdot"))

# format always-on labels
fig.update_layout(xaxis_title=(f"SG/Rd - {str(weeks_prior)} WEEKS PRIOR"))
fig.update_layout(yaxis_title=(f"SG/Rd - {str(weeks_after)} WEEKS AFTER"))
fig.update_layout(yaxis = dict(titlefont = dict(size=15)))
fig.update_layout(xaxis = dict(titlefont = dict(size=15)))
fig.update_layout(legend_title="Datagolf Rank")

# format hover labels
fig.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))
fig.update_traces(hovertemplate=
                    "<b>%{customdata[0]}</b> \
                    <br>Count: %{customdata[1]}</b> \
                    <br>=================</b> \
                    <br>SG Before:%{x:>20}<br>After Loss SG:%{y:>14}</b> \
                    <br><b>Change:%{customdata[3]:>23}</b> \
                    <br>=================</b> \
                    <br>Wins Since 2017:%{customdata[4]:>11}</b> \
                    <br>Data Golf Rank:%{customdata[2]:>13}")
                                                  

fig.show()

In [ ]:
round(custom_chart[custom_chart['count'] >= 1].sort_values(rank_bin),2).columns

In [ ]:
# CHART INPUTS #

stroke_delta = int(1)
weeks_prior = int(20)
weeks_after = int(6)
rank_bin = 'bin_40'

chart_data = data[data.r4_delta <= stroke_delta].reset_index(drop=True)

chart_data['prior_date'] = chart_data['event_completed'] - pd.Timedelta(weeks=weeks_prior)
chart_data['post_date'] = chart_data['event_completed'] + pd.Timedelta(weeks=weeks_after)

stats_cols = ['round_score','sg_putt', 'sg_arg', 'sg_app', 'sg_ott',
       'sg_t2g', 'sg_total', 'driving_dist', 'driving_acc', 'gir',
       'scrambling', 'prox_rgh', 'prox_fw']

#######

losers_rounds = []

for i in np.arange(0, len(chart_data)):

    guy = chart_data.player_name[i]
    prior = str(chart_data.prior_date[i])
    event = str(chart_data.event_completed[i])
    post = str(chart_data.post_date[i])
    
    rounds_before = round(stats
                          [(stats.player_name == guy) & (stats.event_completed > prior) & (stats.event_completed < event)]
                          .groupby('player_name', as_index=False)
                          [stats_cols]
                          .mean(),2)
    
    rounds_after = round(stats
                         [(stats.player_name == guy) & (stats.event_completed < post) & (stats.event_completed > event)]
                         .groupby('player_name', as_index=False)
                         [stats_cols]
                         .mean(),2) 

    # add post_ prefix to columns headers
    rounds_after.columns = 'post_' + rounds_after.columns
    rounds_after.rename(columns={'post_player_name':'player_name'}, inplace=True)
    
    # merge players before and after rounds together and send to list
    all_rounds = pd.merge(rounds_before,rounds_after, on='player_name')
    losers_rounds.append(all_rounds)
    
########

# concat before and after rounds
custom_chart = pd.concat(losers_rounds).reset_index(drop=True)

temp_ranks = dg_rankings[['player_name','datagolf_rank']]
custom_chart = pd.merge(custom_chart, temp_ranks, on='player_name')

# bring in 'count', wins, and world ranking 'bin_100'
temp_count = custom_chart.groupby('player_name',as_index=False)['driving_dist'].count().sort_values('driving_dist', ascending=False).rename(columns={'driving_dist':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_count, on='player_name', how = 'left')
temp_bin = chart_data[['player_name',rank_bin]].drop_duplicates()#.sort_values('loser_key', ascending=False).rename(columns={'loser_key':'count'}).reset_index(drop=True)
custom_chart = custom_chart.merge(temp_bin, on='player_name', how = 'left')
temp_wins = stats[(stats.finish_pos == 1) & (stats.round_num == 4)].groupby('player_name')['finish_pos'].count().sort_values(ascending=False)
custom_chart = custom_chart.merge(temp_wins, on='player_name', how = 'left')
custom_chart.rename(columns={'finish_pos':'career_wins'},inplace=True)
custom_chart['career_wins'] = custom_chart['career_wins'].fillna(0)

temp_m = custom_chart.groupby(rank_bin,as_index=False)[['sg_total', 'post_sg_total']].median()
custom_chart = custom_chart.merge(temp_m, on=rank_bin, how = 'left').rename(columns = 
                                                                                 {'sg_total_x':'sg_total_round', 
                                                                                  'sg_total_y':'sg_total', 
                                                                                  'post_sg_total_x':'post_sg_total_round',
                                                                                  'post_sg_total_y':'post_sg_total',
                                                                                  'delta_sg_total_x':'delta_sg_total_round', 
                                                                                  'delta_sg_total_y':'delta_sg_total'}
                                                                                )
# add sg delta & datagolf rankings columns
custom_chart['delta_sg_total'] = custom_chart['post_sg_total'] - custom_chart['sg_total']
custom_chart['delta_sg_t2g'] = custom_chart['post_sg_t2g'] - custom_chart['sg_t2g']
custom_chart['bubble_size'] = 5


fig = px.scatter(round(custom_chart.sort_values(rank_bin),2),
                 x = 'sg_total',
                 y = 'post_sg_total',
                 title = "SG Before/After Big Loss",
                 template = 'seaborn',
                 color = rank_bin,
                 size = 'bubble_size',
                 size_max = 20,
                 hover_name='player_name',
                 custom_data=['player_name', 'count','datagolf_rank','delta_sg_total','career_wins'],
#                  marginal_x="box",
#                  marginal_y="box",
                 height = 700,
                 width = 950
                )

# format grids and lines
fig.add_hline(y=0,line_width=3, line_color="Black")
fig.add_vline(x=0,line_width=3, line_color="Black")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig.add_shape(type="line", x0=-1.5, y0=-1.5, x1=2.5, y1=2.5, line=dict(color="DarkRed", width=1.5, dash="dashdot"))

# format always-on labels
fig.update_layout(xaxis_title="SG / Rd - 24 weeks prior to 'tough loss'")
fig.update_layout(yaxis_title="SG / Rd - 3 weeks after 'tough loss'")
fig.update_layout(legend_title="World Rank")

# format hover labels
fig.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))
fig.update_traces(hovertemplate=
#                   "<b>%{customdata[0]}</b> \
#                   "<br>Count: %{customdata[1]}</b> \
#                    <br><br>Career Wins: %{customdata[4]}</b> \
#                    <br>World Rank: %{customdata[2]}</b> \
                  "<br><br>Baseline SG:   %{x}<br>After Loss SG:   %{y}</b> \
                  <br><br>delta:  %{customdata[3]}")
fig.show()

In [ ]:
# temp = custom_chart[custom_chart.delta_sg_total]
print("============================")
print(f'Mean:               {round(custom_chart.delta_sg_total.mean(),2)}')
print("============================")
print(f'Median:             {round(custom_chart.delta_sg_total.median(),2)}')
print("============================")
print(f'Std Dev:             {round(custom_chart.delta_sg_total.std(),2)}')
print("============================")
print(f'Variance:            {round(custom_chart.delta_sg_total.var(),2)}')
print("============================")
# custom_chart.delta_sg_total.describe()
#[custom_chart.player_name == 'Henley, Russell'][['player_name', '''sg_total_round', 'post_sg_total_round', 'delta_sg_total_round','sg_total', 'post_sg_total', 'delta_sg_total']]

In [ ]:
# # all instances with means and deltas

# print(custom_chart.shape)
# print('==================')
# print(custom_chart.dtypes)
# print('==================')
# print(custom_chart.columns)
# print('==================')
# round(custom_chart,2)

In [ ]:
# quick look at sg stats before, after, and delta

print("==========================")
print(round(custom_chart['sg_total'].mean(),2))
print("==========================")
print(round(custom_chart['post_sg_total'].mean(),2))
print("==========================")
print(round(custom_chart['delta_sg_total'].mean(),2))
print("==========================")

In [ ]:
fig.write_html("../practice_files/r4_delta_final.html")